In [1]:
import pandas as pd
import numpy as np
import itertools

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")

In [3]:
num = 28000
train_x = train.iloc[:num, 1:].astype("float32").values.reshape(num, 28, 28, 1) / 255
train_y = train.iloc[:num, 0].astype("int32").values
train_y = np_utils.to_categorical(train_y, 10)

test_x = train.iloc[num:, 1:].astype("float32").values.reshape(14000, 28, 28, 1) / 255
test_y = train.iloc[num:, 0].astype("int32").values
test_y = np_utils.to_categorical(test_y, 10)

In [4]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.1))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(300, activation = "relu"))
#model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              loss = "categorical_crossentropy",
              metrics=["accuracy"])

In [5]:
model.fit(train_x, train_y, batch_size=88, epochs=20)

Epoch 1/20
28000/28000 [==============================] - 256s 9ms/step - loss: 0.2557 - acc: 0.9214
Epoch 2/20
28000/28000 [==============================] - 252s 9ms/step - loss: 0.0648 - acc: 0.9797
Epoch 3/20
28000/28000 [==============================] - 251s 9ms/step - loss: 0.0446 - acc: 0.9856
Epoch 4/20
28000/28000 [==============================] - 252s 9ms/step - loss: 0.0328 - acc: 0.9896
Epoch 5/20
28000/28000 [==============================] - 252s 9ms/step - loss: 0.0268 - acc: 0.9915
Epoch 6/20
28000/28000 [==============================] - 253s 9ms/step - loss: 0.0208 - acc: 0.9929
Epoch 7/20
28000/28000 [==============================] - 257s 9ms/step - loss: 0.0199 - acc: 0.9934
Epoch 8/20
28000/28000 [==============================] - 253s 9ms/step - loss: 0.0149 - acc: 0.9948
Epoch 9/20
28000/28000 [==============================] - 254s 9ms/step - loss: 0.0137 - acc: 0.9960
Epoch 10/20
28000/28000 [==============================] - 253s 9ms/step - loss: 0.0124 - a

In [6]:
result = model.evaluate(test_x, test_y)
print('test accuracy: ', result[1])

14000/14000 [==============================] - 50s 4ms/step
test accuracy:  0.9912857142857143


In [7]:
model.save('jtsai_model_test.h5')
submit = pd.read_csv("sample_submission.csv")
new_test_x = pd.read_csv("test.csv")

In [8]:
new_test = new_test_x.astype("float32").values.reshape(num, 28, 28, 1) / 255

In [9]:
# Generate test predictions
preds = model.predict_classes(new_test, verbose=0)

In [10]:
preds

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [11]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "42-jtsai-2.csv")

In [12]:
del model